<a href="https://colab.research.google.com/github/yoouza/2020-dacon-satellite/blob/master/data_analysis/modelling_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("/content/drive/My Drive/CARD_SPENDING_RESIDENT.csv", sep="\t")

jung = df[df.GU_CD == 140]
nowon = df[df.GU_CD == 350]

jung2019 = jung[jung.STD_DD < 20190528]

data = jung2019[jung2019.MCT_CAT_CD == 81] # 음료 식품 카테고리

data = data.groupby('STD_DD').sum()[['USE_CNT','USE_AMT']].reset_index()

amt_X, amt_y = [], []
cnt_X, cnt_y = [], []
for i in range(89):
    amt_X.append(data.USE_AMT[i:i+20].values)
    amt_y.append(data.USE_AMT[i+20:i+27].values)
    cnt_X.append(data.USE_CNT[i:i+20].values)
    # cnt_y.append(data.USE_CNT[i+20:i+27].values)

amt_X = np.array(amt_X)
amt_y = np.array(amt_y)
cnt_X = np.array(cnt_X)
# cnt_y = np.array(cnt_y)

X = np.concatenate([amt_X.reshape(-1, 20, 1), cnt_X.reshape(-1, 20, 1)], axis=2)

X.shape, amt_y.shape

((89, 20, 2), (89, 7))

In [4]:
y = amt_y

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
torch.manual_seed(1015)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

X = torch.FloatTensor(X).to(device)
y = torch.FloatTensor(y).to(device)
# valid_cycle_x = torch.FloatTensor(valid_cycle_x).to(device)
# valid_cycle_y = torch.FloatTensor(valid_cycle_y).to(device)

class LSTMModel_trend(nn.Module):
    def __init__(self, input_size, hidden_size):
        # time model
        super(LSTMModel_trend, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.time_fc = nn.Linear(hidden_size, 7)
        
    def forward(self, x_time):
        # time part
        hidden = (
            torch.zeros(1, x_time.size(0), self.hidden_size).to(device),
            torch.zeros(1, x_time.size(0), self.hidden_size).to(device)
        )
        
        out, _ = self.lstm(x_time, hidden)
        out = self.time_fc(out_time[:, -7:, :])#
                        
        return out.view(-1,7)

#hidden state : 전달과정 중간 출력의 차원
model_trend = LSTMModel_trend(input_size = 3, hidden_size = 32, no_time_size = 3).to(device)
optimizer = torch.optim.Adam(model_trend.parameters(), lr = 1e-2)
criterion = nn.MSELoss(size_average = True)

# Train model
num_epochs  = 200
hist = np.zeros(num_epochs)

# Number of steps to unroll
look_back = 21
seq_dim =look_back - 1  

for t in range(num_epochs):
    train_trend_pred = model_trend(train_trend_x, train_trend_x_notime)
    loss = criterion(train_trend_pred, train_trend_y)

    if t % 10 == 0 and t !=0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

for i in range(10):
    i = i*7
    plt.plot(np.arange(20),train_trend_x.cpu().detach().numpy()[i,:,0],
             marker = 'o', color = 'black', label = 'True_input')
    plt.plot(np.arange(21,28),train_trend_y.cpu().detach().numpy()[i],
             marker = 'o', color = 'red', label = 'True_output', alpha = 0.5)
    plt.plot(np.arange(21,28),train_trend_pred.cpu().detach().numpy()[i],
             color = 'blue', label = 'Predict', marker = 'x', ls = '--', alpha = 0.5)
    plt.legend()
    plt.show()